# Análise de Redes nos Hinos da ICM
Este notebook explora a construção e visualização de redes baseadas em coocorrência de palavras e similaridade entre hinos da coletânea principal da Igreja Cristã Maranata.

O objetivo é identificar padrões de conexão entre palavras e hinos, utilizando grafos para revelar relações semânticas e temáticas.

---
**Conteúdo do notebook:**
- Construção de rede de coocorrência de palavras
- Visualização da rede de palavras
- Construção de rede de similaridade entre hinos
- Visualização da rede de hinos
- Exportação dos grafos para análise externa

Este material é público e pode ser compartilhado para fins de pesquisa, estudo ou divulgação cultural.

# Parte 8 - Redes

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations
from collections import Counter
import re
import pandas as pd

hinos_analise: pd.DataFrame = pd.read_pickle("..\\assets\\hinos_analise_tokens.pkl")


# Supondo que 'df' tem uma coluna "tokens" com listas de palavras
def build_word_cooccurrence_network(df, window_size=2, min_cooccurrence=2):
    cooccurrences = Counter()

    for tokens in df["tokens_no_stops"]:
        for i in range(len(tokens) - window_size + 1):
            window = tokens[i : i + window_size]
            for pair in combinations(window, 2):
                cooccurrences[tuple(sorted(pair))] += 1

    # Cria o grafo
    G = nx.Graph()
    for (w1, w2), freq in cooccurrences.items():
        if freq >= min_cooccurrence:
            G.add_edge(w1, w2, weight=freq)

    return G


G = build_word_cooccurrence_network(hinos_analise, window_size=3, min_cooccurrence=3)

# Visualização simples
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, k=0.5)
nx.draw_networkx_nodes(G, pos, node_size=100, node_color="lightblue")
nx.draw_networkx_edges(G, pos, alpha=0.3)
nx.draw_networkx_labels(G, pos, font_size=9)
plt.title("Rede de Coocorrência de Palavras")
plt.axis("off")
plt.show()

### Construção da Rede de Coocorrência de Palavras
Nesta etapa, criamos um grafo onde os nós representam palavras e as arestas indicam que essas palavras aparecem próximas nos hinos (janela de 3 palavras).
A força da conexão (peso) indica quantas vezes a coocorrência ocorre na coletânea.
Esse tipo de rede ajuda a identificar associações frequentes e temas recorrentes no repertório.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Usa a coluna 'texto' do dataframe
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(hinos_analise["tokens_no_stops"].apply(lambda x: " ".join(x)))

# Calcula similaridade entre hinos
similarity = cosine_similarity(X)

# Cria grafo com base na similaridade
G_docs = nx.Graph()
for i in range(len(hinos_analise)):
    G_docs.add_node(i, label=f"Hino {i+1}")

# Adiciona arestas apenas para hinos com similaridade alta
threshold = 0.3
for i in range(len(hinos_analise)):
    for j in range(i + 1, len(hinos_analise)):
        if similarity[i, j] > threshold:
            G_docs.add_edge(i, j, weight=similarity[i, j])

# Visualização
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G_docs, k=0.6)
nx.draw(G_docs, pos, with_labels=True, node_color="lightgreen", node_size=600)
plt.title("Rede de Similaridade entre Hinos")
plt.show()

### Rede de Similaridade entre Hinos
Agora, criamos uma rede onde cada nó representa um hino e as arestas indicam alta similaridade de conteúdo entre eles, calculada via TF-IDF e similaridade de cosseno.
Apenas hinos com similaridade acima de um limiar são conectados, formando grupos temáticos ou estilísticos.
A visualização permite identificar clusters de hinos com mensagens ou vocabulário semelhantes.

In [ ]:
# Exemplo com seu grafo de coocorrências
nx.write_gexf(G, "rede_palavras.gexf")

# Ou com o grafo de hinos
nx.write_gexf(G_docs, "rede_hinos.gexf")

### Exportação dos Grafos para Análise Externa
Por fim, exportamos as redes geradas (palavras e hinos) em formato GEXF, que pode ser aberto em softwares como Gephi para análises avançadas de redes.
Isso permite explorar métricas de centralidade, modularidade e visualizar comunidades dentro do repertório.